# Temporal Network Basics

In [1]:
import pathpy as pp

In [2]:
tn = pp.TemporalNetwork(directed=True)

In [3]:
tn.add_edge('a', 'b', timestamp=1, color='red')
tn.add_edge('b', 'c', timestamp=2)
tn.add_edge('c', 'd', timestamp=4, color='green')
tn.add_edge('d', 'f', timestamp=4)
tn.add_edge('a', 'f', timestamp=6)
tn.add_edge('c', 'f', start=7, duration=3)
tn.add_edge('b', 'f', start=7, end=11)
print(tn)

Uid:			0x27ca2a05470
Type:			TemporalNetwork
Directed:		True
Multi-Edges:		False
Number of unique nodes:	5
Number of unique edges:	7
Number of temp nodes:	5
Number of temp edges:	7
Observation period:	1 - 11
Observation length:	10


In [4]:
tn.plot()

In [5]:
for start, end, v in tn.tnodes:
    print(start, end, v)

-inf inf a
-inf inf b
-inf inf c
-inf inf d
-inf inf f


In [6]:
for start, end, e in tn.tedges:
    print(start, end, e)
    print(tn.edges[e].attributes)

1 2.0 0x27cc374b320
{(1, 2.0, 'color'): 'red'}
2 3.0 0x27cc3755e48
{}
4 5.0 0x27cc3755518
{(4, 5.0, 'color'): 'green'}
4 5.0 0x27cc3755ba8
{}
6 7.0 0x27ca2978c50
{}
7 11 0x27ca2a275f8
{}
7 inf 0x27ca29789e8
{}


In [7]:
n = pp.Network.from_temporal_network(tn, min_time=0, max_time=100)
n.plot()

In [8]:
n = pp.Network.from_temporal_network(tn, min_time=0, max_time=2)
n.plot()

In [14]:
rolling_window = pp.algorithms.RollingTimeWindow(tn, 1, 1, False)

In [15]:
networks = [n for n in rolling_window]

In [16]:
networks[0].plot()

In [17]:
networks[1].plot()

In [18]:
networks[2].plot()

In [19]:
networks[3].plot()